In [23]:
import requests
from scrapy import Selector
from parsel import Selector
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import numpy as np

from numerize import numerize as nz

from lets_plot import * # This imports all of ggplot2's functions
LetsPlot.setup_html()


In [ ]:
filename = "D:/me204/code/prg1.ipynb/hotels_oyo.csv" # Can you change this to the file you want to read?
df=pd.read_csv(filename)
df.info()

In [ ]:
df=df.drop('Unnamed: 0',axis=1)
df


In [ ]:
df['Destination']=df['Destination'].str.split(',').str[0]
df['Destination']

In [ ]:
df['Offer Percentage']=df['Offer Percentage'].str.split(' ').str[0]
df['Offer Percentage']

In [ ]:
df['Ratings'] = df['Ratings'].str.replace(r'[()]', '', regex=True)
df['Ratings']

In [ ]:
df['Ratings'] = df['Ratings'].str.replace(' Ratings', '', regex=False)
df['Ratings']

In [ ]:
place_to_state = {
    "agra": "Uttar Pradesh", "ahmedabad": "Gujarat", "ajmer": "Rajasthan", "aligarh": "Uttar Pradesh", 
    "alwar": "Rajasthan", "ambala": "Haryana", "amritsar": "Punjab", "asansol": "West Bengal", 
    "aurangabad": "Maharashtra", "ayodhya": "Uttar Pradesh", "bangalore": "Karnataka", "bareilly": "Uttar Pradesh", 
    "barpeta": "Assam", "bathinda": "Punjab", "bawal": "Haryana", "berhampore": "West Bengal", 
    "bhilai": "Chhattisgarh", "bhiwadi": "Rajasthan", "bhubaneswar": "Odisha", "bhopal": "Madhya Pradesh", 
    "bijnor": "Uttar Pradesh", "bilaspur": "Chhattisgarh", "bodhgaya": "Bihar", "bulandshahr": "Uttar Pradesh", 
    "calicut": "Kerala", "chandigarh": "Chandigarh (UT)", "chennai": "Tamil Nadu", "coimbatore": "Tamil Nadu", 
    "coorg": "Karnataka", "cuttack": "Odisha", "dehradun": "Uttarakhand", "delhi": "Delhi", 
    "delhi_transit": "Delhi", "deoghar": "Jharkhand", "dhanbad": "Jharkhand", "dharamshala": "Himachal Pradesh", 
    "digha": "West Bengal", "durgapur": "West Bengal", "ernakulam": "Kerala", "erode": "Tamil Nadu", 
    "faridaba": "Haryana", "gangtok": "Sikkim", "gaya": "Bihar", "ghaziabad": "Uttar Pradesh", 
    "goa": "Goa", "gorakhpur": "Uttar Pradesh", "guntur": "Andhra Pradesh", "gurgaon": "Haryana", 
    "guwahati": "Assam", "gwalior": "Madhya Pradesh", "haridwar": "Uttarakhand", "hisar": "Haryana", 
    "hyderabad": "Telangana", "indore": "Madhya Pradesh", "jabalpur": "Madhya Pradesh", "jaipur": "Rajasthan", 
    "jalandhar": "Punjab", "jammu": "Jammu & Kashmir", "jamshedpur": "Jharkhand", "jodhpur": "Rajasthan", 
    "kakinada": "Andhra Pradesh", "kalimpong": "West Bengal", "kannur": "Kerala", "kanpur": "Uttar Pradesh", 
    "kanyakumari": "Tamil Nadu", "karnal": "Haryana", "katra": "Jammu & Kashmir", "khammam": "Telangana", 
    "kochi": "Kerala", "kodaikanal": "Tamil Nadu", "kolkata": "West Bengal", "kollam": "Kerala", 
    "kota": "Rajasthan", "kovalam": "Kerala", "kurukshetra": "Haryana", "latur": "Maharashtra", 
    "lucknow": "Uttar Pradesh", "ludhiana": "Punjab", "madurai": "Tamil Nadu", "mahabaleshwar": "Maharashtra", 
    "mahabalipuram": "Tamil Nadu", "manali": "Himachal Pradesh", "mandarmoni": "West Bengal", "manesar": "Haryana", 
    "mangalore": "Karnataka", "mathura": "Uttar Pradesh", "mcleod_ganj": "Himachal Pradesh", "meerut": "Uttar Pradesh", 
    "mohali": "Punjab", "moradabad": "Uttar Pradesh", "motihari": "Bihar", "mumbai": "Maharashtra", 
    "mussoorie": "Uttarakhand", "mysore": "Karnataka", "nagpur": "Maharashtra", "namakkal": "Tamil Nadu", 
    "narnaul": "Haryana", "nashik": "Maharashtra", "navi_mumbai": "Maharashtra", "noida": "Uttar Pradesh", 
    "ooty": "Tamil Nadu", "pachmarhi": "Madhya Pradesh", "pathanamthitta": "Kerala", "patiala": "Punjab", 
    "patna": "Bihar", "pondicherry": "Puducherry (UT)", "blair": "Andaman & Nicobar", "pune": "Maharashtra", 
    "puri": "Odisha", "raipur": "Chhattisgarh", "rajahmundry": "Andhra Pradesh", "rajkot": "Gujarat", 
    "ranchi": "Jharkhand", "rewari": "Haryana", "rishikesh": "Uttarakhand", "rohtak": "Haryana", 
    "salem": "Tamil Nadu", "satna": "Madhya Pradesh", "shillong": "Meghalaya", "shimla": "Himachal Pradesh", 
    "shirdi": "Maharashtra", "siliguri": "West Bengal", "sirsa": "Haryana", "sitapur": "Uttar Pradesh", 
    "somnath": "Gujarat", "srinagar": "Jammu & Kashmir", "surat": "Gujarat", "tarapith": "West Bengal", 
    "tirupati": "Andhra Pradesh", "tehri": "Uttarakhand", "thekkady": "Kerala", "thrissur": "Kerala", 
    "tiruppur": "Tamil Nadu", "trichy": "Tamil Nadu", "trivandrum": "Kerala", "udaipur": "Rajasthan", 
    "udupi_manipal": "Karnataka", "ujjain": "Madhya Pradesh", "vadodara": "Gujarat", "varanasi": "Uttar Pradesh", 
    "vellore": "Tamil Nadu", "vijayawada": "Andhra Pradesh", "virudhunagar": "Tamil Nadu", 
    "visakhapatnam": "Andhra Pradesh", "vitthalapur": "Gujarat", "vizianagaram": "Andhra Pradesh", 
    "vrindavan": "Uttar Pradesh", "warangal": "Telangana", "wardha": "Maharashtra", "wayanad": "Kerala", 
    "yercaud": "Tamil Nadu", "zirakpur": "Punjab"
}

# Add the 'State' column based on the 'Place' column
df["State"] = df["Place"].map(place_to_state)

df


In [ ]:
df.rename(columns={"Review": "Describe"}, inplace=True)
df

In [ ]:
df.rename(columns={"Ratings": "review_count"}, inplace=True)
df.info()

In [ ]:
review_to_rating = {
    "Fabulous": 5,
    "Excellent": 4.5,
    "Very Good": 4,
    "Good": 3.5,
    "Fair": 3
}

# Create the new "Ratings" column
df["Ratings"] = df["Describe"].map(review_to_rating)
df

In [ ]:
column_order = ["State","Place", "Hotel Name", "Destination", "Describe", "Ratings", 
                "review_count", "Actual Price", "Offer Percentage", "Price"]

df = df[column_order]
df.info()

In [ ]:
df.to_csv('hotels_rooms_oyo.csv', index=False)

In [ ]:
filename="D:/me204/code/prg1.ipynb/data_file/hotels_rooms_oyo.csv"
df=pd.read_csv(filename)

In [ ]:
df.isna().sum()

State                 0
Place                 0
Hotel Name            0
Destination          35
Describe            248
Ratings             248
review_count        248
Actual Price        116
Offer Percentage    116
Price               116
dtype: int64

In [ ]:
df=df.dropna(subset=['Describe'])
df=df.dropna(subset=['Actual Price'])
df=df.dropna(subset=['Destination'])

In [ ]:
df.isna().sum()

State               0
Place               0
Hotel Name          0
Destination         0
Describe            0
Ratings             0
review_count        0
Actual Price        0
Offer Percentage    0
Price               0
dtype: int64

In [ ]:

df['review_count'] = pd.to_numeric(df['review_count'], errors='coerce').fillna(0).astype(int)

df

,State,Place,Hotel Name,Destination,Describe,Ratings,review_count,Actual Price,Offer Percentage,Price
0,Uttar Pradesh,agra,Hotel O Sai President,Near Rawali Mandir,Good,3.5,453,₹3962,71%,₹932
1,Uttar Pradesh,agra,Hotel O ASA AGRA,Plot number 1 Fatehabad road Plot number 1 Fat...,Excellent,4.5,275,₹2912,72%,₹648
2,Uttar Pradesh,agra,Hotel O Salt And Pepper Hotel And Resturant,Near Prachin Pathwari Mandir,Good,3.5,137,₹4948,71%,₹1137
3,Uttar Pradesh,agra,Hotel O KING PALACE,Near Fatehabad Road,Excellent,4.5,9,₹3378,71%,₹797
4,Uttar Pradesh,agra,Hotel O 804621 Hotel Vintage House,1717 Fatehabad Road,Excellent,4.5,134,₹2912,70%,₹702
...,...,...,...,...,...,...,...,...,...,...
5621,Punjab,zirakpur,Hotel O Elista,India - 140603,Very Good,4.0,138,₹4051,68%,₹1036
5622,Punjab,zirakpur,Hotel O AV Continental,Near Chandigarh Airport Next to New Life Line ...,Fair,3.0,50,₹1797,62%,₹542
5623,Punjab,zirakpur,Hotel O The Banur Heights,Mohali,Good,3.5,35,₹6552,77%,₹1212
5624,Punjab,zirakpur,Collection O Hotel North East,mohali punjab,Very Good,4.0,42,₹6115,72%,₹1414


In [ ]:
df['Actual Price'] = df['Actual Price'].str.replace('₹', '')
df

,State,Place,Hotel Name,Destination,Describe,Ratings,review_count,Actual Price,Offer Percentage,Price
0,Uttar Pradesh,agra,Hotel O Sai President,Near Rawali Mandir,Good,3.5,453,3962,71%,₹932
1,Uttar Pradesh,agra,Hotel O ASA AGRA,Plot number 1 Fatehabad road Plot number 1 Fat...,Excellent,4.5,275,2912,72%,₹648
2,Uttar Pradesh,agra,Hotel O Salt And Pepper Hotel And Resturant,Near Prachin Pathwari Mandir,Good,3.5,137,4948,71%,₹1137
3,Uttar Pradesh,agra,Hotel O KING PALACE,Near Fatehabad Road,Excellent,4.5,9,3378,71%,₹797
4,Uttar Pradesh,agra,Hotel O 804621 Hotel Vintage House,1717 Fatehabad Road,Excellent,4.5,134,2912,70%,₹702
...,...,...,...,...,...,...,...,...,...,...
5621,Punjab,zirakpur,Hotel O Elista,India - 140603,Very Good,4.0,138,4051,68%,₹1036
5622,Punjab,zirakpur,Hotel O AV Continental,Near Chandigarh Airport Next to New Life Line ...,Fair,3.0,50,1797,62%,₹542
5623,Punjab,zirakpur,Hotel O The Banur Heights,Mohali,Good,3.5,35,6552,77%,₹1212
5624,Punjab,zirakpur,Collection O Hotel North East,mohali punjab,Very Good,4.0,42,6115,72%,₹1414


In [ ]:
df['Actual Price'] = pd.to_numeric(df['Actual Price'], errors='coerce').astype(int)
df

,State,Place,Hotel Name,Destination,Describe,Ratings,review_count,Actual Price,Offer Percentage,Price
0,Uttar Pradesh,agra,Hotel O Sai President,Near Rawali Mandir,Good,3.5,453,3962,71%,₹932
1,Uttar Pradesh,agra,Hotel O ASA AGRA,Plot number 1 Fatehabad road Plot number 1 Fat...,Excellent,4.5,275,2912,72%,₹648
2,Uttar Pradesh,agra,Hotel O Salt And Pepper Hotel And Resturant,Near Prachin Pathwari Mandir,Good,3.5,137,4948,71%,₹1137
3,Uttar Pradesh,agra,Hotel O KING PALACE,Near Fatehabad Road,Excellent,4.5,9,3378,71%,₹797
4,Uttar Pradesh,agra,Hotel O 804621 Hotel Vintage House,1717 Fatehabad Road,Excellent,4.5,134,2912,70%,₹702
...,...,...,...,...,...,...,...,...,...,...
5621,Punjab,zirakpur,Hotel O Elista,India - 140603,Very Good,4.0,138,4051,68%,₹1036
5622,Punjab,zirakpur,Hotel O AV Continental,Near Chandigarh Airport Next to New Life Line ...,Fair,3.0,50,1797,62%,₹542
5623,Punjab,zirakpur,Hotel O The Banur Heights,Mohali,Good,3.5,35,6552,77%,₹1212
5624,Punjab,zirakpur,Collection O Hotel North East,mohali punjab,Very Good,4.0,42,6115,72%,₹1414


In [ ]:
df["Price"]=df["Price"].str.replace('₹', '')
df

,State,Place,Hotel Name,Destination,Describe,Ratings,review_count,Actual Price,Offer Percentage,Price
0,Uttar Pradesh,agra,Hotel O Sai President,Near Rawali Mandir,Good,3.5,453,3962,71%,932
1,Uttar Pradesh,agra,Hotel O ASA AGRA,Plot number 1 Fatehabad road Plot number 1 Fat...,Excellent,4.5,275,2912,72%,648
2,Uttar Pradesh,agra,Hotel O Salt And Pepper Hotel And Resturant,Near Prachin Pathwari Mandir,Good,3.5,137,4948,71%,1137
3,Uttar Pradesh,agra,Hotel O KING PALACE,Near Fatehabad Road,Excellent,4.5,9,3378,71%,797
4,Uttar Pradesh,agra,Hotel O 804621 Hotel Vintage House,1717 Fatehabad Road,Excellent,4.5,134,2912,70%,702
...,...,...,...,...,...,...,...,...,...,...
5621,Punjab,zirakpur,Hotel O Elista,India - 140603,Very Good,4.0,138,4051,68%,1036
5622,Punjab,zirakpur,Hotel O AV Continental,Near Chandigarh Airport Next to New Life Line ...,Fair,3.0,50,1797,62%,542
5623,Punjab,zirakpur,Hotel O The Banur Heights,Mohali,Good,3.5,35,6552,77%,1212
5624,Punjab,zirakpur,Collection O Hotel North East,mohali punjab,Very Good,4.0,42,6115,72%,1414


In [ ]:
df['Price'] = pd.to_numeric(df['Price'], errors='coerce').astype(int)
df

,State,Place,Hotel Name,Destination,Describe,Ratings,review_count,Actual Price,Offer Percentage,Price
0,Uttar Pradesh,agra,Hotel O Sai President,Near Rawali Mandir,Good,3.5,453,3962,71%,932
1,Uttar Pradesh,agra,Hotel O ASA AGRA,Plot number 1 Fatehabad road Plot number 1 Fat...,Excellent,4.5,275,2912,72%,648
2,Uttar Pradesh,agra,Hotel O Salt And Pepper Hotel And Resturant,Near Prachin Pathwari Mandir,Good,3.5,137,4948,71%,1137
3,Uttar Pradesh,agra,Hotel O KING PALACE,Near Fatehabad Road,Excellent,4.5,9,3378,71%,797
4,Uttar Pradesh,agra,Hotel O 804621 Hotel Vintage House,1717 Fatehabad Road,Excellent,4.5,134,2912,70%,702
...,...,...,...,...,...,...,...,...,...,...
5621,Punjab,zirakpur,Hotel O Elista,India - 140603,Very Good,4.0,138,4051,68%,1036
5622,Punjab,zirakpur,Hotel O AV Continental,Near Chandigarh Airport Next to New Life Line ...,Fair,3.0,50,1797,62%,542
5623,Punjab,zirakpur,Hotel O The Banur Heights,Mohali,Good,3.5,35,6552,77%,1212
5624,Punjab,zirakpur,Collection O Hotel North East,mohali punjab,Very Good,4.0,42,6115,72%,1414


In [ ]:
df["Offer Percentage"]=df["Offer Percentage"].str.replace('%', '')
df

,State,Place,Hotel Name,Destination,Describe,Ratings,review_count,Actual Price,Offer Percentage,Price
0,Uttar Pradesh,agra,Hotel O Sai President,Near Rawali Mandir,Good,3.5,453,3962,71,932
1,Uttar Pradesh,agra,Hotel O ASA AGRA,Plot number 1 Fatehabad road Plot number 1 Fat...,Excellent,4.5,275,2912,72,648
2,Uttar Pradesh,agra,Hotel O Salt And Pepper Hotel And Resturant,Near Prachin Pathwari Mandir,Good,3.5,137,4948,71,1137
3,Uttar Pradesh,agra,Hotel O KING PALACE,Near Fatehabad Road,Excellent,4.5,9,3378,71,797
4,Uttar Pradesh,agra,Hotel O 804621 Hotel Vintage House,1717 Fatehabad Road,Excellent,4.5,134,2912,70,702
...,...,...,...,...,...,...,...,...,...,...
5621,Punjab,zirakpur,Hotel O Elista,India - 140603,Very Good,4.0,138,4051,68,1036
5622,Punjab,zirakpur,Hotel O AV Continental,Near Chandigarh Airport Next to New Life Line ...,Fair,3.0,50,1797,62,542
5623,Punjab,zirakpur,Hotel O The Banur Heights,Mohali,Good,3.5,35,6552,77,1212
5624,Punjab,zirakpur,Collection O Hotel North East,mohali punjab,Very Good,4.0,42,6115,72,1414


In [ ]:
df['Offer Percentage'] = pd.to_numeric(df['Offer Percentage'], errors='coerce').astype(int)
df

,State,Place,Hotel Name,Destination,Describe,Ratings,review_count,Actual Price,Offer Percentage,Price
0,Uttar Pradesh,agra,Hotel O Sai President,Near Rawali Mandir,Good,3.5,453,3962,71,932
1,Uttar Pradesh,agra,Hotel O ASA AGRA,Plot number 1 Fatehabad road Plot number 1 Fat...,Excellent,4.5,275,2912,72,648
2,Uttar Pradesh,agra,Hotel O Salt And Pepper Hotel And Resturant,Near Prachin Pathwari Mandir,Good,3.5,137,4948,71,1137
3,Uttar Pradesh,agra,Hotel O KING PALACE,Near Fatehabad Road,Excellent,4.5,9,3378,71,797
4,Uttar Pradesh,agra,Hotel O 804621 Hotel Vintage House,1717 Fatehabad Road,Excellent,4.5,134,2912,70,702
...,...,...,...,...,...,...,...,...,...,...
5621,Punjab,zirakpur,Hotel O Elista,India - 140603,Very Good,4.0,138,4051,68,1036
5622,Punjab,zirakpur,Hotel O AV Continental,Near Chandigarh Airport Next to New Life Line ...,Fair,3.0,50,1797,62,542
5623,Punjab,zirakpur,Hotel O The Banur Heights,Mohali,Good,3.5,35,6552,77,1212
5624,Punjab,zirakpur,Collection O Hotel North East,mohali punjab,Very Good,4.0,42,6115,72,1414


In [ ]:

df.to_csv('hotel_booking_web.csv', index=False)